In [1]:
import os
import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.datasets as dataset
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from google.colab import files
import time
import struct
import PIL

In [2]:
import torchvision.models as models
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
gpu_boole = torch.cuda.is_available()
print(gpu_boole)

True


In [4]:
#input_data_dir = "/content/drive/My Drive/tiny-imagenet-200/"

In [12]:
#!wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
#!unzip tiny-imagenet-200.zip



import io
import glob
import os
from shutil import move
from os.path import join
from os import listdir, rmdir

target_folder = './tiny-imagenet-200/val/'
test_folder   = './tiny-imagenet-200/test/'

#os.mkdir(test_folder)
val_dict = {}
with open('./tiny-imagenet-200/val/val_annotations.txt', 'r') as f:
    for line in f.readlines():
        split_line = line.split('\t')
        val_dict[split_line[0]] = split_line[1]
        
paths = glob.glob('./tiny-imagenet-200/val/images/*')
for path in paths:
    file = path.split('/')[-1]
    folder = val_dict[file]
    if not os.path.exists(target_folder + str(folder)):
        os.mkdir(target_folder + str(folder))
        os.mkdir(target_folder + str(folder) + '/images')
    if not os.path.exists(test_folder + str(folder)):
        os.mkdir(test_folder + str(folder))
        os.mkdir(test_folder + str(folder) + '/images')
        
        
for path in paths:
    file = path.split('/')[-1]
    folder = val_dict[file]
    if len(glob.glob(target_folder + str(folder) + '/images/*')) <25:
        dest = target_folder + str(folder) + '/images/' + str(file)
    else:
        dest = test_folder + str(folder) + '/images/' + str(file)
    move(path, dest)
    
rmdir('./tiny-imagenet-200/val/images')

In [13]:
#ImageNet dataloading taken from 
#https://github.com/tjmoon0104/pytorch-tiny-imagenet/blob/master/AlexNet.ipynb
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
    'test': transforms.Compose([
        transforms.ToTensor(),
    ])
}

data_dir = 'tiny-imagenet-200/'
num_workers = {
    'train' : 100,
    'val'   : 0,
    'test'  : 0
}
image_datasets = {x: dataset.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x])
                  for x in ['train', 'val','test']}
dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=100,
                                             shuffle=True, pin_memory =True)
              for x in ['train', 'val', 'test']}

ValueError: ignored

In [7]:
print(dataset_sizes)

for i, data in enumerate(dataloaders['test']):
  images, labels = data
  print(labels)

{'train': 100000, 'val': 10000, 'test': 10000}


KeyError: ignored

In [76]:
netparam = models.resnet18(pretrained=True)
netparam.avgpool = nn.AdaptiveAvgPool2d(1)
netparam.fc.out_features = 200
print(netparam.parameters)
optimizer = torch.optim.SGD(netparam.parameters(), lr=0.001, momentum=0.9)
#optimizer = torch.optim.Adam(netparam.parameters(), 0.001)
loss_metric = torch.nn.CrossEntropyLoss()
if gpu_boole:
 netparam = netparam.cuda()

<bound method Module.parameters of ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)


In [78]:
def train_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['train']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()
        
    if verbose:
        print('Train accuracy: %f %%' % (100.0 * correct / total))
        print('Train loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    
def test_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['test']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()    
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()

    if verbose:
        print('Test accuracy: %f %%' % (100.0 * correct / total))
        print('Test loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total

def validation_eval(netparam, verbose = 1):
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['val']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()
        
    if verbose:
        print('Validation accuracy: %f %%' % (100.0 * correct / total))
        print('Validation loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    '''
    correct = 0
    total = 0
    loss_sum = 0
    for images, labels in dataloaders['val']:
        if gpu_boole:
            images, labels = images.cuda(), labels.cuda()
        outputs = netparam(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted.float() == labels.float()).sum()
        loss_sum += loss_metric(outputs,labels).detach()

    if verbose:
        print('Validation accuracy: %f %%' % (100.0 * correct / total))
        print('Validation loss: %f' % (loss_sum.cpu().data.numpy().item() / total))

    return 100.0 * correct / total, loss_sum.cpu().data.numpy().item() / total
    '''

In [79]:
epochs = 10

#defining batch train loss recording arrays for later visualization/plotting:
loss_batch_store = []
train_accuracy_scores = []
test_accuracy_scores = []
validation_accuracy_scores = []
training_loss = []
epoch_list = [i for i in range(1, epochs + 1)]
epoch_loss = []
maxvalacc = 0
print("Starting Training")
#training loop:
for epoch in range(epochs):
  time1 = time.time() #timekeeping
  for i, (x, y) in enumerate(dataloaders['train']):
    if (i % 100 ==0 ):
      print(i)
    if gpu_boole:
      x = x.cuda()
      y = y.cuda()
    if i > 0 or epoch > 0:
      optimizer.zero_grad()
    outputs = netparam.forward(x)
    loss = loss_metric(outputs,y)
    loss.backward()
    if i > 0 or epoch > 0:
      loss_batch_store.append(loss.cpu().data.numpy().item())
    optimizer.step()
    epoch_loss.append(loss.detach())
    

  print("Epoch",epoch+1,':')
  train_perc, train_loss = train_eval(netparam)
  valid_perc, valid_loss = validation_eval(netparam)
  test_perc, test_loss = test_eval(netparam)
  #if valid_perc > maxvalacc:
   # maxvalacc = valid_perc
    #torch.save(netparam.state_dict(), path) 
  validation_accuracy_scores.append(valid_perc)
  time2 = time.time() #timekeeping
  print('Elapsed time for epoch:',time2 - time1,'s')
  print('ETA of completion:',(time2 - time1)*(epochs - epoch - 1)/60,'minutes')
  print()

## Plotting batch-wise train loss curve:
#plt.plot(epoch_list, training_loss, '-o', label = 'train_loss', color = 'blue')
#plt.xlabel('Epoch Number')
#plt.ylabel('Training Loss at Given Epoch')
#plt.legend()
#plt.show()
#plt.plot(epoch_list, train_accuracy_scores, color = 'blue')
#plt.plot(epoch_list, test_accuracy_scores, color = 'blue')
#plt.plot(epoch_list, validation_accuracy_scores, color = 'blue')
#plt.xlabel('Epoch Number')
#plt.ylabel('Accuracy in Percent')
test_perc_a, test_loss_a = test_eval(netparam)

Starting Training
0
100
200
300
400
500
600
700
800
900
Epoch 1 :
Train accuracy: 21.507999 %
Train loss: 0.034577
Validation accuracy: 20.820000 %
Validation loss: 0.035319
Test accuracy: 1.086667 %
Test loss: 0.071743
Elapsed time for epoch: 103.23199486732483 s
ETA of completion: 15.484799230098725 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 2 :
Train accuracy: 40.653000 %
Train loss: 0.024531
Validation accuracy: 34.700001 %
Validation loss: 0.027121
Test accuracy: 0.920000 %
Test loss: 0.085914
Elapsed time for epoch: 102.96472096443176 s
ETA of completion: 13.728629461924235 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 3 :
Train accuracy: 50.870998 %
Train loss: 0.019693
Validation accuracy: 41.320000 %
Validation loss: 0.024477
Test accuracy: 0.866667 %
Test loss: 0.091611
Elapsed time for epoch: 102.54199886322021 s
ETA of completion: 11.963233200709025 minutes

0
100
200
300
400
500
600
700
800
900
Epoch 4 :
Train accuracy: 58.651997 %
Train loss: 0.016327
Val

KeyboardInterrupt: ignored